<a href="https://colab.research.google.com/github/CSID-DGU/2023-2-OSSP1-CrashLandingOnYou-6/blob/master/foodanalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 18.7 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp310-cp310-linux_x86_64.whl size=661677 sha256=0224e902508e673668e6d84a9d4e22937d93ed5facefad5afd57bb17c8101a77
  Stored in directory: /root/.cache/pip/wheels/1a/1e/0d/99f55911d90f2b95b9f7c176d5813ef3622894a4b30fde6bd3
Successfully built gluonnlp
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 61.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 39.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 99.4 MB/s eta 0:00:00
     ━━━━━

In [ ]:
# ★
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-pejhq4pd/kobert-tokenizer_7ea416ac4d224d2496acccd5b5919e7d
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-pejhq4pd/kobert-tokenizer_7ea416ac4d224d2496acccd5b5919e7d
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4632 sha256=96f2b1ee3a7a21ee1c1c68992aff67905e4e460f25fd87c36b60bd13d771e67e
  Stored in directory: /tmp/pip-ephem-wheel-cache-trcc2lmq/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from kobert_tokenizer import KoBERTTokenizer
import pandas as pd
import numpy as np
import json

In [ ]:
# Set device to CPU if CUDA is not available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the tokenizer and model
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
#model = BertForSequenceClassification.from_pretrained('skt/kobert-base-v1', num_labels=12)
# 드롭아웃 추가 (모델 구조 변경 필요)
model = BertForSequenceClassification.from_pretrained('skt/kobert-base-v1', num_labels=12, hidden_dropout_prob=0.3)

# Move the model to the appropriate device
model.to(device)


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at skt/kobert-base-v1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.3, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, 

In [ ]:
# 레이블 인코딩
label_encoder = LabelEncoder()
labels = ['볶음', '무침', '굽기', '끓이기', '비빔', '찜', '삶기', '회', '튀김', '조림','부침','기타']
label_encoder.fit(labels)


LabelEncoder()

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# 여러 JSON 파일들로부터 데이터를 로드하고 하나의 데이터셋으로 결합하는 함수
def load_and_combine_json(file_paths):
    combined_texts = []
    combined_labels = []
    for file_path in file_paths:
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
            texts = [item['recipe'] for item in data]
            labels = [item['classification'] for item in data]
            combined_texts.extend(texts)
            combined_labels.extend(labels)
    return combined_texts, combined_labels

# 각 레이블에 해당하는 JSON 파일 경로들
file_paths = [
    '/content/drive/MyDrive/볶음.json',
    '/content/drive/MyDrive/무침.json',
    '/content/drive/MyDrive/굽기.json',
    '/content/drive/MyDrive/끓이기.json',
    '/content/drive/MyDrive/비빔.json',
    '/content/drive/MyDrive/찜.json',
    '/content/drive/MyDrive/삶기.json',
    '/content/drive/MyDrive/회.json',
    '/content/drive/MyDrive/튀김.json',
    '/content/drive/MyDrive/조림.json',
    '/content/drive/MyDrive/부침.json',
    '/content/drive/MyDrive/기타.json',


]

# 데이터 로드 및 결합
texts, labels = load_and_combine_json(file_paths)

# 레이블 인코딩
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

# 이후 과정 (토크나이징, 데이터셋 및 로더 생성, 학습 등)은 동일하게 유지합니다



In [ ]:
# 훈련 데이터와 검증 데이터로 분할
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)

In [ ]:
# 리스트의 각 요소를 하나의 문자열로 변환
train_texts = [' '.join(recipe) for recipe in train_texts]
val_texts = [' '.join(recipe) for recipe in val_texts]

# 토크나이징 및 데이터셋 생성
train_encodings = tokenizer(train_texts, truncation=True, padding=True, return_tensors='pt', max_length=512)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, return_tensors='pt', max_length=512)


In [ ]:
# 커스텀 데이터셋 클래스
class RecipeDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
train_dataset = RecipeDataset(train_encodings, train_labels)
val_dataset = RecipeDataset(val_encodings, val_labels)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

optimizer = AdamW(model.parameters(), lr=1e-5)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_loader) * 3)


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.train()
    for epoch in range(3):  # 에포크 설정
        for batch in train_loader:
            optimizer.zero_grad()
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            scheduler.step()
        validate(model, val_loader, device)

def validate(model, val_loader, device):
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            val_loss += outputs.loss.item()
    # 평균 검증 손실 계산
    val_loss /= len(val_loader)
    print(f'Validation Loss: {val_loss}')

# 모델 학습 실행
train(model, optimizer, train_loader, val_loader, scheduler, device)

Validation Loss: 2.4978842735290527
Validation Loss: 2.4980669021606445
Validation Loss: 2.495846748352051


In [ ]:
import random

# 데이터셋 로드
texts, labels = load_and_combine_json(file_paths)

# 데이터셋에서 무작위로 하나의 레시피 선택
random_index = random.randint(0, len(texts) - 1)
random_recipe = texts[0]

print(f"선택된 레시피: {random_recipe}")


선택된 레시피: ['양배추 씻어 주고 채 썰어주세요.', '양파를 다져주세요.', '참치캔에 기름을 버려주세요.', '팬에 식용유 3스푼을 둘러 양배추를 볶아주세요.중불', '양파를 넣고 같이 볶아주세요. 양파가 투명하게 변할때까지 볶아주세요.', '굴소스 1/2스푼과 참치액젓 1/2스푼을 넣어주세요.', '기름뺀 참치를 넣고 잘 섞어가며 볶아주세요.', '가운데 홈을 만들어주고 계란을 깨트려 넣어주세요.계란이 익으면 불을 꺼주세요.', '그릇에 밥 1공기를 덜어놓고 위에 양배추 볶음을 올려주세요. 마지막으로 참기름 1/2스푼과 참깨 약간을 뿌려주면 완성이에요.']


In [ ]:
# 모델과 토크나이저 저장 경로 설정
model_save_path = "/content/drive/My Drive/Model/model"
tokenizer_save_path = "/content/drive/My Drive/Model/tokenizer"

# 모델 저장
model.save_pretrained(model_save_path)

# 토크나이저 저장
tokenizer.save_pretrained(tokenizer_save_path)

('/content/drive/My Drive/Model/tokenizer/tokenizer_config.json',
 '/content/drive/My Drive/Model/tokenizer/special_tokens_map.json',
 '/content/drive/My Drive/Model/tokenizer/spiece.model',
 '/content/drive/My Drive/Model/tokenizer/added_tokens.json')

In [ ]:
# 예측하고자 하는 새로운 레시피
new_recipe = " 팬에 식용유를 두 바퀴 정도 두르고 대파와 당근을 볶다가 당근이 노릇해지면 어묵과 양파를 넣고 볶습니다어묵이 노릇노릇하게 볶아질 때 양념장을 넣고 양념장이 골고루 베어 들도록 중약불에서 약 3분간 섞어가며 볶아주면 완성	"
encoded_new_recipe = tokenizer.encode_plus(
    new_recipe,
    max_length=512,
    truncation=True,
    padding='max_length',
    return_tensors='pt'
)

# 모델에 입력하기 위해 토크나이징된 데이터를 CUDA로 이동(사용 가능한 경우)
input_ids = encoded_new_recipe['input_ids'].to(device)
attention_mask = encoded_new_recipe['attention_mask'].to(device)

# 예측
model.eval()  # 평가 모드로 설정
with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_mask)
    predictions = torch.argmax(outputs.logits, dim=-1)

# 결과 해석
predicted_label = label_encoder.inverse_transform(predictions.cpu().numpy())[0]
print(f"예측된 레시피 분류: {predicted_label}")

예측된 레시피 분류: 볶음
